In [1]:
using PastaQ
using ITensors

In [2]:
# number of qubits
N = 5

# manually create a circuit to prepare GHZ state,
# or use built-in call `circuit = ghz(n)` 
circuit = Tuple[("H", 1)]
for j in 1:N-1
  push!(circuit, ("CX", (j, j+1)))
end

# run the circuit to obtain the output MPS
hilbert = qubits(N)
ψ = runcircuit(hilbert, circuit)


# sample projective measurements in the computational basis
@show getsamples(ψ, 5)

getsamples(ψ, 5) = [0 0 0 0 0; 1 1 1 1 1; 1 1 1 1 1; 0 0 0 0 0; 1 1 1 1 1]


5×5 Matrix{Int64}:
 0  0  0  0  0
 1  1  1  1  1
 1  1  1  1  1
 0  0  0  0  0
 1  1  1  1  1

In [3]:
ψ = runcircuit(hilbert, circuit)
@which getsamples(ψ, 5)

getsamples(M0::Union{MPO, MPS}, nshots::Int64; kwargs...) in PastaQ at /Users/alcalde/.julia/packages/PastaQ/D5CCg/src/circuits/getsamples.jl:154

In [4]:
ψ = runcircuit(hilbert, circuit)
ρ = outer(ψ, prime(ψ))
@which sample(ρ)

sample(M::MPO) in ITensors at /local/alcalde/workprojects/forks/installed/ITensors.jl/src/mps/mpo.jl:920

In [5]:
noisemodel = (1 => ("depolarizing", (p = 0.1,)), 
              2 => ("depolarizing", (p = 0.5,)))

circuit2 = Tuple[("H", 1)]
for j in 1:N-1
  push!(circuit2, ("CX", (j, j+1)))
end

ψ = randomMPS(hilbert, linkdims=10)

runcircuit(ψ, circuit2; noismodel=noisemodel)

ψ = randomMPS(hilbert, linkdims=10)
runcircuit(ψ, circuit2; noise = noisemodel)

MPO
[1] ((dim=2|id=673|"Qubit,Site,n=1")', (dim=2|id=673|"Qubit,Site,n=1"), (dim=4|id=323|"Link,n=1"))
[2] ((dim=2|id=549|"Qubit,Site,n=2")', (dim=4|id=323|"Link,n=1"), (dim=2|id=549|"Qubit,Site,n=2"), (dim=16|id=872|"Link,n=1"))
[3] ((dim=2|id=465|"Qubit,Site,n=3")', (dim=16|id=872|"Link,n=1"), (dim=2|id=465|"Qubit,Site,n=3"), (dim=16|id=977|"Link,n=1"))
[4] ((dim=2|id=199|"Qubit,Site,n=4")', (dim=16|id=977|"Link,n=1"), (dim=2|id=199|"Qubit,Site,n=4"), (dim=4|id=582|"Link,n=1"))
[5] ((dim=4|id=582|"Link,n=1"), (dim=2|id=29|"Qubit,Site,n=5")', (dim=2|id=29|"Qubit,Site,n=5"))


In [6]:
# define a noise model with different error rates for
# one- and two-qubit gates
noisemodel = (1 => ("depolarizing", (p = 0.1,)), 
              2 => ("depolarizing", (p = 0.5,)))

# run a noisy circuit
ρ = runcircuit(hilbert, circuit; noise = noisemodel)
@show fidelity(ψ, ρ)
@show getsamples(ρ, 5)

fidelity(ψ, ρ) = 0.02717837060776039
getsamples(ρ, 5) = [1 1 0 0 0; 1 0 0 0 1; 1 0 0 0 0; 0 0 0 0 0; 1 1 1 0 0]


5×5 Matrix{Int64}:
 1  1  0  0  0
 1  0  0  0  1
 1  0  0  0  0
 0  0  0  0  0
 1  1  1  0  0

In [7]:
# quantum processes can also be obtained.
# unitary MPO
U = runcircuit(circuit; process = true)
# Choi matrix
Λ = runcircuit(circuit; process = true, noise = noisemodel)

MPO
[1] ((dim=2|id=180|"Output,Qubit,Site,n=1")', (dim=2|id=180|"Input,Qubit,Site,n=1")', (dim=2|id=180|"Input,Qubit,Site,n=1"), (dim=2|id=180|"Output,Qubit,Site,n=1"), (dim=5|id=509|"Link,n=1"))
[2] ((dim=2|id=473|"Output,Qubit,Site,n=2")', (dim=5|id=509|"Link,n=1"), (dim=2|id=473|"Input,Qubit,Site,n=2")', (dim=2|id=473|"Input,Qubit,Site,n=2"), (dim=2|id=473|"Output,Qubit,Site,n=2"), (dim=5|id=667|"Link,n=1"))
[3] ((dim=2|id=827|"Output,Qubit,Site,n=3")', (dim=5|id=667|"Link,n=1"), (dim=2|id=827|"Input,Qubit,Site,n=3")', (dim=2|id=827|"Input,Qubit,Site,n=3"), (dim=2|id=827|"Output,Qubit,Site,n=3"), (dim=5|id=484|"Link,n=1"))
[4] ((dim=2|id=432|"Output,Qubit,Site,n=4")', (dim=5|id=484|"Link,n=1"), (dim=2|id=432|"Input,Qubit,Site,n=4")', (dim=2|id=432|"Input,Qubit,Site,n=4"), (dim=2|id=432|"Output,Qubit,Site,n=4"), (dim=5|id=198|"Link,n=1"))
[5] ((dim=5|id=198|"Link,n=1"), (dim=2|id=884|"Output,Qubit,Site,n=5")', (dim=2|id=884|"Input,Qubit,Site,n=5")', (dim=2|id=884|"Input,Qubit,Site,n=